In [2]:
!pip install --upgrade tensorflow_hub
!pip install tensorflow_text

     -------------------------------------- 90.3/90.3 kB 851.3 kB/s eta 0:00:00
     ---------------------------------------- 5.0/5.0 MB 4.2 MB/s eta 0:00:00
     ---------------------------------------- 24.4/24.4 MB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 5.9 MB/s eta 0:00:00
     -------------------------------------- 895.7/895.7 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("Suicide_Detection.csv")

In [10]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [48]:
df_test = df.sample(10000, random_state=42)

In [18]:
df_test["class"].value_counts()

non-suicide    15043
suicide        14957
Name: class, dtype: int64

In [19]:
df_test.head()

,text,class
74414,I Don't know?7? Months self harm free and the ...,suicide
149516,I HAVE TO START BECOMING RICH I HAVE TO START ...,non-suicide
12484,"A poem (haiku) for u/Me-Game-Dev hi, hello hel...",non-suicide
14043,I've honestly got no idea what to do anymore.I...,suicide
30673,Do you ever just cry? Like you just think abou...,non-suicide


In [49]:
df_test["suicide"] = np.where(df_test["class"] == "suicide", 1, 0)

In [24]:
df_test.head()

,text,class,suicide
74414,I Don't know?7? Months self harm free and the ...,suicide,1
149516,I HAVE TO START BECOMING RICH I HAVE TO START ...,non-suicide,0
12484,"A poem (haiku) for u/Me-Game-Dev hi, hello hel...",non-suicide,0
14043,I've honestly got no idea what to do anymore.I...,suicide,1
30673,Do you ever just cry? Like you just think abou...,non-suicide,0


In [25]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [26]:
bert_preprocess = hub.KerasLayer(preprocess_url)
bert_encoder = hub.KerasLayer(encoder_url)

In [27]:
def sentence_embedding(sentences):
    """cümleyi embedding yapan fonksiyon
    
    pooled_output: Girilen cümlenin gömülü vektörü 768 length
    
    
    """
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)["pooled_output"]

In [37]:
e = sentence_embedding(["banana",
                       "grapes",
                       "mango",
                       "jeff bezos",
                       "elon musk",
                       "bill gates"])

In [38]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[-0.76069176, -0.14219391,  0.49604627, ...,  0.42165327,
        -0.5322141 ,  0.80312157],
       [-0.8602321 , -0.21242921,  0.49156946, ...,  0.3979805 ,
        -0.60506266,  0.84471637],
       [-0.7128863 , -0.15463915,  0.38401607, ...,  0.35278717,
        -0.50991344,  0.73474115],
       [-0.82533467, -0.355506  , -0.5906974 , ..., -0.01613783,
        -0.61417574,  0.87230283],
       [-0.75041354, -0.26812634, -0.26689735, ...,  0.02839332,
        -0.59381   ,  0.7974988 ],
       [-0.78544366, -0.29949698,  0.41027397, ...,  0.5222541 ,
        -0.4957355 ,  0.8150751 ]], dtype=float32)>

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[4]], [e[5]])

array([[0.9164159]], dtype=float32)

In [50]:
# Bert layers

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural Network Layers

layer = tf.keras.layers.Dropout(0.1, name="dropout")(outputs["pooled_output"])
layer = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(layer)

# final model
model = tf.keras.Model(inputs=[text_input], outputs=[layer])

In [51]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                

In [52]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [53]:
X = df_test["text"]
y = df_test["suicide"]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [55]:
model.fit(X_train, y_train, epochs=10, batch_size=1000)

Epoch 1/10
8/8 [==============================] - 596s 74s/step - loss: 0.6474 - accuracy: 0.6323 - precision: 0.6034 - recall: 0.7673
Epoch 2/10
8/8 [==============================] - 591s 74s/step - loss: 0.6200 - accuracy: 0.6877 - precision: 0.7049 - recall: 0.6438
Epoch 3/10
8/8 [==============================] - 593s 74s/step - loss: 0.5968 - accuracy: 0.7222 - precision: 0.7033 - recall: 0.7670
Epoch 4/10
8/8 [==============================] - 591s 74s/step - loss: 0.5792 - accuracy: 0.7333 - precision: 0.7216 - recall: 0.7578
Epoch 5/10
8/8 [==============================] - 590s 74s/step - loss: 0.5603 - accuracy: 0.7492 - precision: 0.7404 - recall: 0.7660
Epoch 6/10
8/8 [==============================] - 591s 74s/step - loss: 0.5496 - accuracy: 0.7548 - precision: 0.7428 - recall: 0.7778
Epoch 7/10
8/8 [==============================] - 590s 74s/step - loss: 0.5373 - accuracy: 0.7632 - precision: 0.7491 - recall: 0.7903
Epoch 8/10
8/8 [==============================] - 592s 

In [113]:
len(X_test)

2000

In [58]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 182s 3s/step - loss: 0.5031 - accuracy: 0.7980 - precision: 0.7825 - recall: 0.8021


[0.5031064748764038,
 0.7979999780654907,
 0.7825203537940979,
 0.8020833134651184]

In [59]:
model.save('base_model.keras')

In [60]:
y_pred = model.predict(X_test)

63/63 [==============================] - 186s 3s/step


In [62]:
y_pred

array([[0.6908078 ],
       [0.17345595],
       [0.7064164 ],
       ...,
       [0.7734686 ],
       [0.70560515],
       [0.52503836]], dtype=float32)

In [75]:
df[df["class"] == "suicide"]["text"].sample(1).values

array(['Could use a hugEveryday i look at mom’s sweet happy face and think of how much i want to die ..every morning i wake up with the burden of being alive , all i want is to disappear completely but it’s only gonna destroy her. I’m very young yet so damn lonely, i really wish i can be heard'],
      dtype=object)

In [76]:
tahmin = ["Could use a hugEveryday i look at mom’s sweet happy face and think of how much i want to die ..every morning i wake up with the burden of being alive , all i want is to disappear completely but it’s only gonna destroy her. I’m very young yet so damn lonely, i really wish i can be heard"]

model.predict(tahmin)

1/1 [==============================] - 0s 131ms/step


array([[0.7890031]], dtype=float32)

In [77]:
df[df["class"] == "suicide"]["text"].sample(1).values

array(['Did you ever realize?That nobody in your life actually gives a fuck about you? Not one person genuinely cares or makes an effort of any way towards your relationship or friendship? That while you’re sure they would cry if you killed yourself, they would soon get over it and move on, because despite the love and connection you feel towards them, nobody feels that way towards you? Despite the lengths you would go for anybody and everybody at all, nobody would so much as cross a shallow puddle to make you feel welcomed on this earth. \n\nCause yeah folks, that sucks.'],
      dtype=object)

In [80]:
tahmin2 = ["Did you ever realize?That nobody in your life actually gives a fuck about you? Not one person genuinely cares or makes an effort of any way towards your relationship or friendship? That while you’re sure they would cry if you killed yourself, they would soon get over it and move on, because despite the love and connection you feel towards them, nobody feels that way towards you? Despite the lengths you would go for anybody and everybody at all, nobody would so much as cross a shallow puddle to make you feel welcomed on this earth. \n\nCause yeah folks, that sucks."]
model.predict(tahmin2)

1/1 [==============================] - 0s 141ms/step


array([[0.7196343]], dtype=float32)

In [85]:
df[df["class"] == "non-suicide"]["text"].sample(1).values

array(['How to get a girlfriend (from a girl) now that you’re here...\n\nget your ass off of reddit and do your homework'],
      dtype=object)

In [86]:
tahmin3 = ["How to get a girlfriend (from a girl) now that you’re here...\n\nget your ass off of reddit and do your homework"]
model.predict(tahmin3)

1/1 [==============================] - 0s 143ms/step


array([[0.3742782]], dtype=float32)

In [91]:
tahmin4 = ["I've never been happy. I never had real friends that didn't take advantage of me or abuse me. Never had hobbies I enjoyed. I've never enjoyed music, I don't enjoy food. Being around other people and animals make me feel worse. Nothing helps. Everything I do or try just makes me feel worse. I hate living, I hate life. I hate existence as a whole. I haven't gone a day in years without thinking about how much I wish I didn't exist. The only person who never gave up was my mom, but she's not equipped for this. I've tried therapy, I've tried medication. I've tried finding hobbies but nothing sticks. Nothing is enjoyable. I hate living like this, but I have no experience feeling anything else. I dropped out of middle school, and I don't want to continue my education. I don't even want to be a part of society. I wish I was never born. Fuck everything"]
model.predict(tahmin4)

1/1 [==============================] - 0s 151ms/step


array([[0.7776913]], dtype=float32)

In [95]:
tahmin4 = ["I hate my life because I dont have any friends, my parents do not love me and i think i am a disgust person. Why should I live?"]
model.predict(tahmin4)

1/1 [==============================] - 0s 138ms/step


array([[0.47899464]], dtype=float32)

In [100]:
tahmin5 = ["I am so happy right now. I feel like nothing . My life is goin well :)"]
model.predict(tahmin5)

1/1 [==============================] - 0s 164ms/step


array([[0.40010777]], dtype=float32)

In [97]:
tahmin6 = ["im not really living. im hanging on by a thread. the heading is true and i havent washed my hair in longer. im disgusting and i hate myself so much. my hair is a matted mess and i want to shave my head so bad but ive shaved my head before to get rid of lice and i looked terrible for a year while it grew back. i just want to die so i dont have to deal with it. these feeling always pass after a few weeks but idk if i can survive this time. it sucks too because nothing is going wrong and its my dogs 4th birthday in a week and a half and we got a house after living in the worst apartment ever for years. i just dont know what to do. i stopped going to school in 8th grade so i have no future. im trying to get ssi (disability for ppl who havent worked basically) but i know they wont accept me because mental health isnt taken seriously so i dont have any money. im sorry this is probably all over the place im just typing. i just want to die. i feel it would help my family so much because i eat so much food. ugh idk why im doing this i guess im trying to keep myself occupied so i dont do anything. bye i guess"]
model.predict(tahmin6)

1/1 [==============================] - 0s 150ms/step


array([[0.5841863]], dtype=float32)

In [98]:
tahmin7 = ["my boyfriend, best friend, love of my life committed suicide last month. and i genuinely feel like i can’t be here without him. like i’m not supposed to be. everyone else who grieves him has a partner to go to, they feel like they’re supposed to keep living and figure this out. but i don’t. he was my everything. i don’t want this life anymore. everyone is sad for me but they truly do not understand. i struggled before i met him but when i met him it was like this is it. he’s the reason i’m here. and with him gone i have no reason. i keep praying for some car accident or illness or event to happen to me to take me because then no one can say i did it to myself. i feel like i’m being held hostage here for everyone else’s sake. and no one knew me like him so they don’t understand that this is something i’ve struggled with for years and years. his leaving is just an anvil of pain on top of everything. i feel like i’m terminally ill and they’re keeping me plugged up so they don’t have to deal with my death yet. the few people who do check on me offer help but i’m past help. nothing comforts me."]
model.predict(tahmin7)

1/1 [==============================] - 0s 145ms/step


array([[0.76421094]], dtype=float32)

In [101]:
tahmin8 = ["my boyfriend, best friend, love of my life committed suicide last month. and i genuinely feel like i can’t be here without him. like i’m not supposed to be. everyone else who grieves him has a partner to go to, they feel like they’re supposed to keep living and figure this out. but i don’t. he was my everything. i don’t want this life anymore. everyone is sad for me but they truly do not understand. i struggled before i met him but when i met him it was like this is it. he’s the reason i’m here. and with him gone i have no reason. i keep praying for some car accident or illness or event to happen to me to take me because then no one can say i did it to myself. i feel like i’m being held hostage here for everyone else’s sake. and no one knew me like him so they don’t understand that this is something i’ve struggled with for years and years. his leaving is just an anvil of pain on top of everything. i feel like i’m terminally ill and they’re keeping me plugged up so they don’t have to deal with my death yet. the few people who do check on me offer help but i’m past help. nothing comforts me."]
model.predict(tahmin8)

1/1 [==============================] - 0s 161ms/step


array([[0.76421094]], dtype=float32)

In [102]:
tahmin8 = ["I've never been happy. I never had real friends that didn't take advantage of me or abuse me. Never had hobbies I enjoyed. I've never enjoyed music, I don't enjoy food. Being around other people and animals make me feel worse. Nothing helps. Everything I do or try just makes me feel worse. I hate living, I hate life. I hate existence as a whole. I haven't gone a day in years without thinking about how much I wish I didn't exist. The only person who never gave up was my mom, but she's not equipped for this. I've tried therapy, I've tried medication. I've tried finding hobbies but nothing sticks. Nothing is enjoyable. I hate living like this, but I have no experience feeling anything else. I dropped out of middle school, and I don't want to continue my education. I don't even want to be a part of society. I wish I was never born. Fuck everything."]
model.predict(tahmin8)

1/1 [==============================] - 0s 164ms/step


array([[0.7776913]], dtype=float32)

In [109]:
tahmin9 = ["I dont want to continue like this, life has no meaning , i cannot handle all these stress, i am a failure, i think i may be dead in one month"]
model.predict(tahmin9)

1/1 [==============================] - 0s 172ms/step


array([[0.6552784]], dtype=float32)